In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.read_csv("/content/Normalized_Datasets/fe_zscore.csv")

X = df.drop("Target", axis=1)
y = df["Target"]

if y.dtype == 'O':
    y = LabelEncoder().fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

param_grid = {
    'n_neighbors': list(range(3, 21, 2)),  # try k = 3, 5, ..., 19
    'weights': ['uniform', 'distance']
}

grid = GridSearchCV(KNeighborsClassifier(), param_grid, cv=5, scoring='f1_macro', n_jobs=-1)
grid.fit(X_train, y_train)

# Best KNN model
best_knn = grid.best_estimator_
print("Best KNN Parameters:", grid.best_params_)

y_pred = best_knn.predict(X_test)

print("\n Classification Report:\n", classification_report(y_test, y_pred))
print(" Confusion Matrix:")
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cmap='Blues')
plt.title("KNN - Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()

# Optional summary metrics
acc = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='macro')
print(f" Accuracy: {acc:.4f}")
print(f" F1 Score (macro): {f1:.4f}")
